<a href="https://colab.research.google.com/github/ormayraz/ormayraz.github.io/blob/master/Ceal_Analysis_File_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **R&D Experiments Analysis File - Process Analysis**

## Getting Started

The following file includes the gathering and analysis of the process's experimental data.

## Key information
Core part of this analysis file is the usage of the PHREEQC software. This analsis file is hosted by Google Colab which allows for only internet connection in order to operate this file without any prior download or computer refinements.

### **Run the next block for operating this file**

## Useful references

* Phreeqc user guide: https://wwwbrr.cr.usgs.gov/projects/GWC_coupled/phreeqc/html/final.html
* Phreeqc Basic interpreter (useful in the PUNCH block): https://wwwbrr.cr.usgs.gov/projects/GWC_coupled/phreeqc/phreeqc3-html/phreeqc3-54.htm
*   Comparison of thermodynamic data files for PHREEQC: https://www.sciencedirect.com/science/article/abs/pii/S0012825221003895



**This file has been made by Or Mayraz. \\
For any request or additional information please contact:
orm@ceal.earth**


In [ ]:
# -----------------------
# ---- RUN ME FIRST! ----
# -----------------------

# Add libraries
!pip install phreeqpy

# Import libraries
import ipywidgets as widgets
import urllib.request
import phreeqpy.iphreeqc.phreeqc_dll as phreeqc_mod
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, HTML

# Download the IPhreeqc Library and database (the .dll extension is only to make the file appear on the Google Colab directory)
urllib.request.urlretrieve('https://github.com/rispr/phreeqc_web/blob/main/Iphreeqc_compiled/libiphreeqc-3.7.3.so?raw=true', 'libiphreeqc.dll')
lib = '/content/libiphreeqc.dll'

# Function to load and read the database
def get_database(dbName):
  urllib.request.urlretrieve('https://raw.githubusercontent.com/rispr/phreeqc_web/main/database/'+ dbName, dbName)
  # If not using Google Colab the directory will be different
  dbase = '/content/' + dbName
  return dbase

# Function to read and run the input Iphreeqc script
def runPhreeqcSim(lib, dbase, pqc_input):
    phreeqc = phreeqc_mod.IPhreeqc(lib)
    phreeqc.load_database(dbase)
    phreeqc.run_string(pqc_input)
    phreeqc.set_output_file_on()
    return phreeqc


# Seawater composition database
Selection of seawater to be used in the analysis can be viewed here

## Choose Seawater Composition for The Analysis



In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the seawater compositions
seawater_compositions = {
    "IEC - Bat Galim reference ICP": {
        "B": 4, "Ca": 400, "Cl": 19247.3,
        "K": 480, "Mg": 1400, "Na": 9416,
        "S": 892, "Sr": 8
    },
    "Seawater_B": {
        "B": 3.5, "Ca": 390, "Cl": 18500,
        "K": 450, "Mg": 1350, "Na": 9300,
        "S": 850, "Sr": 7.8
    },
}

# Create a dropdown widget with the default composition set to "IEC - Bat Galim reference ICP"
composition_dropdown = widgets.Dropdown(
    options=list(seawater_compositions.keys()),
    value="IEC - Bat Galim reference ICP",
    description='Select Composition:'
)

# This dictionary will store the currently selected composition's ion values.
selected_composition = {}

def on_composition_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        comp_name = change['new']
        comp_values = seawater_compositions[comp_name]
        selected_composition.clear()
        selected_composition.update(comp_values)
        print(f"You selected: {comp_name}")
        for ion, val in comp_values.items():
            print(f"  {ion}: {val}")

# Observe changes in the dropdown
composition_dropdown.observe(on_composition_change, names='value')
display(composition_dropdown)

# Automatically load the default composition into selected_composition
selected_composition.clear()
selected_composition.update(seawater_compositions[composition_dropdown.value])
print("Default composition loaded:")
for ion, val in selected_composition.items():
    print(f"  {ion}: {val}")


Dropdown(description='Select Composition:', options=('IEC - Bat Galim reference ICP', 'Seawater_B'), value='IE…

Default composition loaded:
  B: 4
  Ca: 400
  Cl: 19247.3
  K: 480
  Mg: 1400
  Na: 9416
  S: 892
  Sr: 8


# Defining the experimental conditions and setup

# Calcium Hardness Measurements
## 3500-Ca B. EDTA Titrimetric Method
### General
When EDTA is added to water containing both calcium and
magnesium, it combines first with the calcium. Calcium can
be determined directly, with EDTA, when the pH is made
sufficiently high that the magnesium is largely precipitated as
the hydroxide and an indicator is used that combines with
calcium only.
### Procedure
1.   Dilute the sample to by taking 2 mL of sample together with 48 mL of DI water.
2.   Add 1 mL of NaOH 1M to elevate the pH to 12-13 and precipitate all of the Mg$^{2+}$ ions to Mg(OH)$_2$.
3. Add the HNB indicator. The solution should turn wine red.
4. Titrate the EDTA 0.1N solution until the solution turns from wine red to blue. This indicates the end of the titration point.

### Analysis
$Ca^{2+} [\frac{mg}{L}] = \frac{A⋅B⋅400.8}{V_{sample}}$
\
Where $A$ is mL titrant for sample; $B$ mg CaCO$_3$ equivalent to 1.00 mL EDTA titrant at the calcium indicator endpoint. Which in our case is equal to 1.

### Further reading
For additional information about this analytic method. Please read further in "3500-Ca CALCIUM. in Standard Methods For the Examination of Water and Wastewater (American Public Health Association, 2017). doi:10.2105/SMWW.2882.052."





In [ ]:
def calculate_ca_hardness(A, volume=50, dilution=0.04, B=1):

    if volume <= 0:
        raise ValueError("Total volume of diluted sample must be greater than zero.")
    if dilution <= 0 or dilution > 1:
        raise ValueError("Dilution factor must be between 0 and 1.")

    ca_hardness = (A * B * 400.8) / (volume * dilution)
    return ca_hardness

# In a Google Colab cell you can run this block directly.
# You can use interactive inputs or define the values directly.
try:
    # Using input() to capture user values. You can also set these variables directly.
    A = float(input("Enter the volume of titrant used (A) in mL: "))

    volume_input = input("Enter the total volume of the diluted sample in mL (default 50): ")
    volume = float(volume_input) if volume_input.strip() != "" else 50

    dilution_input = input("Enter the dilution factor (fraction of original sample in diluted solution) (default 0.04): ")
    dilution = float(dilution_input) if dilution_input.strip() != "" else 0.04

    # Calculate and save the calcium concentration
    calcium_concentration = calculate_ca_hardness(A, volume, dilution)

    # Print the result
    print(f"Calcium hardness in the original seawater: {calcium_concentration:.2f} mg/L")
except ValueError as ve:
    print("Input error:", ve)
except Exception as e:
    print("An error occurred:", e)


Enter the volume of titrant used (A) in mL: 2.15
Enter the total volume of the diluted sample in mL (default 50): 
Enter the dilution factor (fraction of original sample in diluted solution) (default 0.04): 
Calcium hardness in the original seawater: 430.86 mg/L


# Alkalinity and pH Measurements

*   Alkalinity is measured in here using an automatic titrator and an analysis tool that is based on Gran Titration. The values are in units of $\frac{meq}{kg}$
*   pH is measured using Thermo scientific pH meter.



In [ ]:
# Prompt the user for the alkalinity measurement and the solution pH, then store them in variables.
try:
    alkalinity_value = float(input("Enter the alkalinity measurement value (meq/kg solution): "))
    pH_value = float(input("Enter the pH value: "))
    print(f"Alkalinity value saved: {alkalinity_value} meq/kg solution")
    print(f"pH value saved: {pH_value}")
except ValueError as ve:
    print("Input error:", ve)
except Exception as e:
    print("An error occurred:", e)


Enter the alkalinity measurement value (meq/kg solution): 1.644
Enter the pH value: 8.72
Alkalinity value saved: 1.644 meq/kg solution
pH value saved: 8.72


# Dissolved Inorganic Carbon (DIC) Analysis

This script allows you to analyze based on your input parameters of seawater composition, pH, temperature and alkalinity, the amount of dissolved inorganic carbon species that are present in the sample.

In [ ]:
dbase = get_database('llnl.dat')  # Load the PHREEQC database file 'llnl.dat'

pqc_input_str = """
SOLUTION 1
    temp            %s
    pH              %s
    pe              4
    redox           pe
    units           mg/kgs
    density         1
    Alkalinity      %s meq/kg solution
    B               %s
    Ca              %s
    Cl              %s
    K               %s
    Mg              %s
    Na              %s
    S               %s
    Sr              %s

SELECTED_OUTPUT
  -reset false
  -high_precision true
  -charge_balance true
  -pe true
  -pH true
  -alkalinity true
  -ionic_strength true
  -temperature true
  -totals     C(4) Ca
  -saturation_indices Calcite Brucite

USER_PUNCH
  -headings rho(kg/L) SC(μS/cm)
  -start
    1 PUNCH RHO
    2 PUNCH SC
  -end
"""

# Initialize the PHREEQC interface
phreeqc = phreeqc_mod.IPhreeqc(lib)
phreeqc.load_database(dbase)

# -------------------------------------------------------------------------
def runSim(phreeqc, temp, pH_val, alkalinity, composition_dict, ca):
    # Extract the ion values from the selected composition dictionary.
    B_val  = composition_dict["B"]
    Cl_val = composition_dict["Cl"]
    K_val  = composition_dict["K"]
    Mg_val = composition_dict["Mg"]
    Na_val = composition_dict["Na"]
    S_val  = composition_dict["S"]
    Sr_val = composition_dict["Sr"]

    # Build the final PHREEQC input string using the template.
    # Order: temp, pH, alkalinity, B, Br, Ca, Cl, K, Mg, Na, S, Sr.
    pqc_input = pqc_input_str % (
        temp, pH_val, alkalinity,
        B_val, ca,      # Ca is provided externally!
        Cl_val, K_val, Mg_val, Na_val, S_val, Sr_val
    )

    # Run the simulation.
    phreeqc_sim = runPhreeqcSim(lib, dbase, pqc_input)

    # Convert the selected output array into a DataFrame.
    data = pd.DataFrame(phreeqc_sim.get_selected_output_array())
    data.columns = data.iloc[0]  # Use the first row as column headings
    data = data[1:]             # Remove the header row from the data
    display(data)
    return data

# -------------------------------------------------------------------------
# Define interactive parameter inputs for temperature and pH (Google Colab slider style)
temp_value = 21.5 # @param {"type":"slider","min":10,"max":30,"step":0.1}
# pH_value = 9.26    # @param {type:"slider", min:0, max:14, step:0.01}

# -------------------------------------------------------------------------
if not selected_composition:
    print("Please select a composition from the dropdown above.")
else:
    # Run the PHREEQC simulation using the selected composition.
    data = runSim(phreeqc, temp_value, pH_value, alkalinity_value, selected_composition, calcium_concentration)

    # Example: Compute DIC concentration from the output.
    # (This assumes that the output DataFrame contains the columns 'C(4)(mol/kgw)' and 'rho(kg/L)'.)
    if 'C(4)(mol/kgw)' in data.columns and 'rho(kg/L)' in data.columns:
        DIC_concentration = data['C(4)(mol/kgw)'] * data['rho(kg/L)']  # in mol/L
        print(f"DIC concentration [mM]: {DIC_concentration.iloc[-1] * 1000:.3f}")
        SI_calcite = data['si_Calcite']
        print(SI_calcite.iloc[-1])


,pH,pe,temp(C),Alk(eq/kgw),mu,charge(eq),C(4)(mol/kgw),Ca(mol/kgw),si_Calcite,si_Brucite,rho(kg/L),SC(μS/cm)
1,8.72,4.0,21.5,0.001698,0.612228,-0.004528,0.001223,0.011105,0.889946,-0.889152,1.033023,0.0


DIC concentration [mM]: 1.263
0.8899464012742517


## Appending To The Log File
The following function takes the values measured and analyzed in this file and appends them into the experimental log book

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import os
from datetime import datetime

# -----------------------------
# Create a DatePicker widget for selecting the date
date_picker = widgets.DatePicker(
    description='Select Date',
    disabled=False
)

# Create a Text widget for entering the Experiment ID/description
experiment_id_text = widgets.Text(
    value='',
    placeholder='Enter experiment description/ID',
    description='Experiment ID:',
    disabled=False
)

# Create a Button widget to trigger saving the data
submit_button = widgets.Button(
    description='Save Experiment Data',
    button_style='success'
)

def on_submit_button_clicked(b):
    # Get the selected date and experiment description from the widgets
    selected_date = date_picker.value
    experiment_id = experiment_id_text.value.strip()

    if selected_date is None:
        print("Please select a date from the calendar.")
        return
    if not experiment_id:
        print("Please enter an experiment ID/description.")
        return

    # Format the date as a string (YYYY-MM-DD)
    date_str = selected_date.strftime('%Y-%m-%d')

    # -----------------------------
    # Extract DIC concentration from the simulation output (if available)
    if 'C(4)(mol/kgw)' in data.columns and 'rho(kg/L)' in data.columns:
        DIC_series = data['C(4)(mol/kgw)'] * data['rho(kg/L)']  # mol/L
        DIC_value = DIC_series.iloc[-1] * 1000  # convert to mM
    else:
        DIC_value = None
        print("Warning: DIC columns not found in the simulation output.")

    # Extract SI_calcite (assumed column name "Calcite")
    if 'si_Calcite' in data.columns:
        SI_calcite = data['si_Calcite'].iloc[-1]
    else:
        SI_calcite = None
        print("Warning: SI for Calcite not found in the simulation output.")

    # -----------------------------
    # Create a dictionary for the experiment row using simulation and input values
    experiment_row = {
        'Date': date_str,
        'Experiment_ID': experiment_id,
        'Temperature': temp_value,
        'pH': pH_value,
        'Ca_measured [ppm]': calcium_concentration,
        'Alkalinity': alkalinity_value,
        'DIC_concentration [mM]': DIC_value,
        'SI_Calcite': SI_calcite
    }

    # Convert the dictionary into a DataFrame (single row)
    experiment_df = pd.DataFrame([experiment_row])

    # Define the Excel filename to store experiment data
    excel_filename = "/content/drive/MyDrive/Colab Notebooks/exp_analysis_process.xlsx"

    # If the file exists, append; otherwise, create a new Excel file.
    if os.path.exists(excel_filename):
        existing_df = pd.read_excel(excel_filename)
        updated_df = pd.concat([existing_df, experiment_df], ignore_index=True)
        updated_df.to_excel(excel_filename, index=False)
    else:
        experiment_df.to_excel(excel_filename, index=False)

    print(f"Experiment data saved to '{excel_filename}'.")

# Connect the button click event to the function
submit_button.on_click(on_submit_button_clicked)

# Display the widgets so you can interactively select the date and enter your experiment ID
display(date_picker, experiment_id_text, submit_button)


DatePicker(value=None, description='Select Date')

Text(value='', description='Experiment ID:', placeholder='Enter experiment description/ID')

Button(button_style='success', description='Save Experiment Data', style=ButtonStyle())

Experiment data saved to 'exp._analysis_process.xlsx'.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Analysis Visualization
This following section access the experimental results analysis log book and visualize it

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import matplotlib.gridspec as gridspec
import textwrap
import numpy as np
import matplotlib.cm as cm

# Set Matplotlib parameters for a Nature‐style figure.
plt.rcParams.update({
    'font.size': 18,
    'font.family': 'serif',
    'axes.labelsize': 20,
    'axes.titlesize': 22,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'axes.linewidth': 1.5,
    'xtick.major.width': 1.5,
    'ytick.major.width': 1.5
})

# Global variable to store the current figure.
current_fig = None

# Load the Excel file.
excel_filename = "/content/drive/MyDrive/Colab Notebooks/exp_analysis_process.xlsx"
df = pd.read_excel(excel_filename)

# Convert the Date column to datetime.
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Create a multi-select widget for experiment IDs.
experiment_options = df["Experiment_ID"].unique().tolist()
experiments_selector = widgets.SelectMultiple(
    options=experiment_options,
    value=tuple(experiment_options),  # Default: all experiments selected.
    description='Experiments',
    disabled=False
)

# Create DatePicker widgets for start and end dates.
min_date = df['Date'].min().date()
max_date = df['Date'].max().date()

start_date_picker = widgets.DatePicker(
    description='Start Date',
    value=min_date
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    value=max_date
)

def update_plots(selected_experiments, start_date, end_date):
    global current_fig
    # Filter the DataFrame by selected experiments and date range.
    filtered_df = df[df["Experiment_ID"].isin(selected_experiments)]
    if start_date is not None:
        filtered_df = filtered_df[filtered_df['Date'] >= pd.to_datetime(start_date)]
    if end_date is not None:
        filtered_df = filtered_df[filtered_df['Date'] <= pd.to_datetime(end_date)]

    # For the bar plots, aggregate by experiment (using the mean) and take the earliest date.
    bar_df = filtered_df.groupby("Experiment_ID", as_index=False).agg({
        "Ca_measured": "mean",
        "DIC_concentration (mM)": "mean",
        "Date": "min"
    })

    # Reorder the aggregated DataFrame according to the order of selection.
    order = list(selected_experiments)
    bar_df['order'] = bar_df['Experiment_ID'].apply(lambda x: order.index(x))
    bar_df = bar_df.sort_values('order')

    # Create a figure using GridSpec:
    # Top panel: Twin-axis bar plot for Calcium and DIC.
    # Bottom panel: Scatter plot for SI vs. pH.
    fig = plt.figure(figsize=(20, 16), facecolor='white')
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1])

    # Top panel: Twin-axis bar plot.
    ax1 = fig.add_subplot(gs[0])
    ax2 = ax1.twinx()

    x = np.arange(len(bar_df))
    width = 0.4  # Bar width.

    bars_calcium = ax1.bar(x - width/2, bar_df["Ca_measured"], width=width,
                           color='skyblue', edgecolor='black', label='Calcium (ppm)')
    bars_dic = ax2.bar(x + width/2, bar_df["DIC_concentration (mM)"], width=width,
                       color='salmon', edgecolor='black', label='DIC (mM)')

    # Create x-axis labels using both Experiment_ID and Date.
    x_labels = [f"{textwrap.fill(row['Experiment_ID'], width=10)}\n{row['Date'].strftime('%Y-%m-%d')}"
                for idx, row in bar_df.iterrows()]
    ax1.set_xticks(x)
    ax1.set_xticklabels(x_labels, rotation=0, ha='center')

    ax1.set_title("Calcium and Dissolved Inorganic Carbon (DIC)", fontsize=22)
    ax1.set_xlabel("Experiment ID & Date", fontsize=20)
    ax1.set_ylabel("Calcium (ppm)", fontsize=20)
    ax2.set_ylabel("DIC (mM)", fontsize=20)
    ax1.tick_params(axis='x', labelsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)

    for bar in bars_calcium:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2, height, f'{height:.1f}',
                 ha='center', va='bottom', fontsize=14)
    for bar in bars_dic:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2, height, f'{height:.1f}',
                 ha='center', va='bottom', fontsize=14)

    # Combine legends and move them outside the plot.
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    legend = ax1.legend(handles1 + handles2, labels1 + labels2, title="Parameter",
                        fontsize=14, title_fontsize=16, frameon=True,
                        bbox_to_anchor=(1.17, 1), loc='upper right')

    # Bottom panel: Scatter plot for SI vs. pH.
    ax3 = fig.add_subplot(gs[1])
    unique_experiments = filtered_df["Experiment_ID"].unique()
    colors = cm.rainbow(np.linspace(0, 1, len(unique_experiments)))
    color_dict = dict(zip(unique_experiments, colors))

    for exp in unique_experiments:
        group = filtered_df[filtered_df["Experiment_ID"] == exp]
        ax3.scatter(group["pH"], group["SI_Calcite"], s=150, color=color_dict[exp],
                    label=exp, edgecolors='black')

    ax3.set_title("Saturation Index (SI) vs pH", fontsize=22)
    ax3.set_xlabel("pH", fontsize=20)
    ax3.set_ylabel("SI", fontsize=20)
    ax3.tick_params(axis='x', labelsize=16)
    ax3.tick_params(axis='y', labelsize=16)
    ax3.legend(title="Experiment ID", fontsize=16, title_fontsize=18, frameon=True)

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # Ensure x-axis labels fit properly
    current_fig = fig  # Store the current figure for saving later.
    plt.show()

# Create an interactive widget linking experiment and date selections to the update function.
interactive_plot = widgets.interactive(update_plots,
                                         selected_experiments=experiments_selector,
                                         start_date=start_date_picker,
                                         end_date=end_date_picker)
display(interactive_plot)

# Button to save the current figure as a PNG file.
save_fig_btn = widgets.Button(
    description='Save Figure as PNG',
    button_style='info'
)

def save_figure(b):
    global current_fig
    if current_fig is not None:
        filename = "/content/drive/MyDrive/Colab Notebooks/Figure_exp_analysis_process.png"
        current_fig.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved as '{filename}'.")
    else:
        print("No figure available to save.")

save_fig_btn.on_click(save_figure)
display(save_fig_btn)


interactive(children=(SelectMultiple(description='Experiments', index=(0, 1, 2, 3, 4, 5, 6), options=('Exp.1 -…

Button(button_style='info', description='Save Figure as PNG', style=ButtonStyle())

Figure saved as '/content/drive/MyDrive/Colab Notebooks/Figure_exp_analysis_process.png'.
